## Imports

In [1]:
import pandas as pd

In [2]:
day = "07"

In [3]:
df = pd.read_csv(f"input_j{day}.txt", header=None, sep=" ")
df.columns = ["hand", "bid"]
new_df = df.copy()

In [4]:
df["hand"] = df["hand"].apply(lambda hand: [x for x in hand])
df

,hand,bid
0,"[A, A, T, K, J]",840
1,"[2, 7, A, 8, 3]",251
2,"[6, T, T, 8, T]",113
3,"[Q, 6, T, 6, T]",693
4,"[9, 9, K, 8, 9]",553
...,...,...
995,"[T, T, T, 4, 2]",614
996,"[9, 9, 5, 9, 5]",384
997,"[7, 4, 4, 7, 4]",412
998,"[J, 5, 7, 4, 5]",293


## Part 1

In [5]:
importances_cartes = {
    "2": "A",
    "3": "B",
    "4": "C",
    "5": "D",
    "6": "E",
    "7": "F",
    "8": "G",
    "9": "H",
    "T": "I",
    "J": "J",
    "Q": "K",
    "K": "L",
    "A": "M"
}

In [6]:
df["departage"] = df.hand.apply(lambda hand: "".join([importances_cartes[x] for x in hand]))

In [7]:
# High card
df['value'] = df['hand'].apply(lambda hand: 1 if len(set(hand)) == len(hand) else 0)

In [8]:
# One pair
df['value'] = df.apply(lambda row: 2 if len(set(row.hand)) == (len(row.hand) - 1) else row.value, axis=1)

In [9]:
# Two pairs
df['value'] = df.apply(lambda row: 3 if len(set(row.hand)) == (len(row.hand) - 2) else row.value, axis=1)

In [10]:
# Three of a kind
df['value'] = df.apply(
    lambda row: 4 if any([
        row.hand.count(element) == 3 for element in importances_cartes.keys()
    ]) else row.value, axis=1)

In [11]:
# Full house
df['value'] = df.apply(
    lambda row: 5 if any([
        (row.hand.count(element) == 3 and len(set(row.hand)) == 2) for element in importances_cartes.keys()
    ]) else row.value, axis=1)

In [12]:
# Four of a kind
df['value'] = df.apply(
    lambda row: 6 if any([
        row.hand.count(element) == 4 for element in importances_cartes.keys()
    ]) else row.value, axis=1)

In [13]:
# Five of a kind
df['value'] = df.apply(lambda row: 7 if len(set(row.hand)) == 1 else row.value, axis=1)

In [14]:
df.sort_values(by=["value", "departage"], inplace=True)
df.reset_index(inplace=True, drop=True)
df["ranking"] = df.index + 1
df

,hand,bid,departage,value,ranking
0,"[2, 3, 9, A, 4]",91,ABHMC,1,1
1,"[2, 4, 3, Q, 8]",127,ACBKG,1,2
2,"[2, 4, A, 8, K]",407,ACMGL,1,3
3,"[2, 6, K, Q, J]",641,AELKJ,1,4
4,"[2, 7, A, 8, 3]",251,AFMGB,1,5
...,...,...,...,...,...
995,"[A, A, A, 2, A]",709,MMMAM,6,996
996,"[A, A, A, 4, A]",878,MMMCM,6,997
997,"[A, A, A, 5, A]",276,MMMDM,6,998
998,"[A, A, A, A, T]",571,MMMMI,6,999


In [15]:
sum(df.bid * df.ranking)

250347426

## Part 2

In [16]:
new_importances_cartes = {
    "J": "A",
    "2": "B",
    "3": "C",
    "4": "D",
    "5": "E",
    "6": "F",
    "7": "G",
    "8": "H",
    "9": "I",
    "T": "J",
    "Q": "K",
    "K": "L",
    "A": "M"
}

In [17]:
new_df["departage"] = new_df.hand.apply(lambda hand: "".join([new_importances_cartes[x] for x in hand]))

In [18]:
def replace_joker(hand):  # All possible hands with jokers will be tested out
    new_hands = {hand}

    for i in range(len(hand)):
        blocked_new_hands = new_hands.copy()
        for new_hand in blocked_new_hands:
            for element in new_importances_cartes.keys():
                new_hands.add(new_hand.replace("J", element, 1))

    new_hands = [[x for x in new_hand] for new_hand in new_hands]
    return new_hands

In [19]:
new_df["hand"] = new_df.hand.apply(replace_joker)

In [20]:
# High card
new_df['value'] = new_df['hand'].apply(
    lambda hands: 1 if any([len(set(hand)) == len(hand) for hand in hands]) else 0
)

In [21]:
# One pair
new_df['value'] = new_df.apply(
    lambda row: 2 if any([len(set(hand)) == (len(hand) - 1) for hand in row.hand]) else row.value,
    axis=1
)

In [22]:
# Two pairs
new_df['value'] = new_df.apply(
    lambda row: 3 if any([len(set(hand)) == (len(hand) - 2) for hand in row.hand]) else row.value,
    axis=1
)

In [23]:
# Three of a kind
new_df['value'] = new_df.apply(
    lambda row: 4 if any(
        [hand.count(element) == 3 for element in new_importances_cartes.keys() for hand in row.hand]) else row.value,
    axis=1
)

In [24]:
# Full house
new_df['value'] = new_df.apply(
    lambda row: 5 if any(
        [hand.count(element) == 3 and len(set(hand)) == 2 for element in new_importances_cartes.keys() for hand in row.hand]) else row.value,
    axis=1
)

In [25]:
# Four of a kind
new_df['value'] = new_df.apply(
    lambda row: 6 if any(
        [hand.count(element) == 4 for element in new_importances_cartes.keys() for hand in row.hand]) else row.value,
    axis=1
)

In [26]:
# five of a kind
new_df['value'] = new_df.apply(
    lambda row: 7 if any(
        [len(set(hand)) == 1 for hand in row.hand]) else row.value,
    axis=1
)

In [27]:
new_df.sort_values(by=["value", "departage"], inplace=True)
new_df.reset_index(inplace=True, drop=True)
new_df["ranking"] = df.index + 1
new_df

,hand,bid,departage,value,ranking
0,"[[2, 3, 9, A, 4]]",91,BCIMD,1,1
1,"[[2, 4, 3, Q, 8]]",127,BDCKH,1,2
2,"[[2, 4, A, 8, K]]",407,BDMHL,1,3
3,"[[2, 7, A, 8, 3]]",251,BGMHC,1,4
4,"[[2, 9, A, 8, 4]]",562,BIMHD,1,5
...,...,...,...,...,...
995,"[[T, 2, T, T, T], [T, 4, T, T, T], [T, J, T, T...",672,JAJJJ,7,996
996,"[[Q, J, Q, Q, Q], [Q, 7, Q, Q, Q], [Q, T, Q, Q...",804,KAKKK,7,997
997,"[[Q, Q, Q, 4, 3], [Q, Q, Q, 5, T], [Q, Q, Q, 2...",795,KKKAA,7,998
998,"[[K, Q, J, K, K], [K, 5, 8, K, K], [K, K, 9, K...",768,LAALL,7,999


In [28]:
sum(new_df.bid * new_df.ranking)

251224870